In [ ]:
%load_ext autoreload
import os, sys
sys.path.append(os.path.abspath('..'))

from IPython.display import Markdown
from IPython.display import Latex
import adaptive
import dependencies.adaptive_tools as adaptive_tools
adaptive.notebook_extension()
import holoviews as hv
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import cmath
import scipy.constants
from multiprocessing import Pool
from functools import partial
import kwant

import supercurrent
import symmetry
import spectrum
import topology
import sns_system
import distributed_sns
import hpc05
import plotting_results
from copy import copy
import warnings

def print_table(table):
    n = len(table)
    m = len(table[0])
    
    res = '| '
    for col in range(m):
        res += table[0][col] + ' |'
    res += '\n|' + m * '----|'
    
    
    for row in range(1, n):
        assert(len(table[row]) == m)
        res += '\n| '
        for col in range(m):
            res += str(table[row][col]) + ' |'
            
    return Markdown(res)

def print_latex(*strings):
    _str = ''
    for string in strings:
        _str += string
    
    return Latex(_str)

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(100, folder='~/two_dim_majoranas', timeout=1800)

# Base system parameters

## System parameters
| Variable | Description | Quantity | Unit |
|------------------------------------------|
| Ll | Width of left superconductor | Length | nm |
| Lr | Width of right superconductor | Length | nm |
| Lm | Width of normal section | Length | nm |
| Ly | Height of system | Length | nm |
| a | Lattice spacing | Length | nm |

## Basic material parameters
| Variable | Description | Quantity | Unit |
|------------------------------------------|
| g_factor | Magnetic g factor of material | dimensionless | - |
| mu | Chemical potential | Energy | meV |
| alpha | Spin orbit coupling strength | spin-momentum coupling strength | meV nm |
| Delta | Superconducting gap| Energy | meV |
| B | In-plane magnetic field strength along stripe direction(y) | Magnetic field | T |
| phase | Relative phase between superconductors | Radians | - |

# Standard Parameters

In [ ]:
syst_pars_standard =  {'Ll' : 500,
                       'Lr' : 500,
                       'Lm' : 250,
                       'Ly' : 4000,
                       'a' :  10}

params_raw_standard = dict(g_factor = 10,
                           mu = 1,
                           alpha = 28,
                           Delta = .18,
                           B = .2,
                          phase = 1,
                          T=0.6)

params_standard = dict(**constants,
                       **params_raw_standard)

# Build system with leads along stripe direction

In [ ]:
syst = sns_system.make_sns_system(**syst_pars_standard)

_ = plotting_results.plot_syst(syst_pars_standard, params_standard, 250, num_lead_cells=4)
_ = plt.xlabel('x')
_ = plt.ylabel('y')

# Calculation of Thouless Energy
\begin{align}
    E_\text{Th} = TN\delta
    , && \delta = \frac{\hbar^2 \pi^2}
                  {2 m_e \left( 2 L_m \right)^2}
\end{align}

Where $T$ is the transmission of the junction (for $\mu$ small 1, see transmission notebook), and $N$ is the number of transverse modes, which is to be calculated.

In [ ]:
def E_thouless(_syst_pars, _params, assume_unit_transmission=True):
    # delta
    def delta(W):
        return (constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*W)**2))

    params = _params.copy()
    params['Delta'] = 0
    
    # Transmission through junction
    syst_pars = _syst_pars.copy()
    syst_pars['Ll'] = syst_pars['Lr'] = 0
    
    if assume_unit_transmission:
        T=1
    else:
        syst_junction = sns_system.make_junction(**syst_pars)
        smat_junction = kwant.smatrix(syst_junction, energy=0, params=params)
        T = smat_junction.transmission(0, 1) / smat_junction.num_propagating(0)

    
    # Number of transversal modes
    syst_pars['Ly'] = syst_pars['a']
    
#     syst = sns_system.make_sns_system(**syst_pars)
#     lead = syst.leads[0]
#     modes = lead.modes(params=params)[0]
#     N = modes.block_nmodes[0] 
    kf = np.sqrt(2*constants['m_eff']*params['mu'])/constants['hbar']
    g_s = 8
    N = g_s * np.ceil(syst_pars['Lm']*kf/np.pi)
    
    return (T, T * N * delta(syst_pars['Lm']))

# Set of junction widths

In [ ]:
Lm = [250, 500, 1000, 2000, 5000]

#### Calculate Thouless Energy for range of junction widths:

In [ ]:
Eth = dict()
p = Pool(len(Lm))

def f(Lm):
    _params = params_standard.copy()
    _params['B'] = 0.1
    
    _syst_pars = syst_pars_standard.copy()
    
    _syst_pars['a'] = 25
    _syst_pars['Lm'] = Lm
    return E_thouless(_syst_pars, _params)

res = p.map(f, Lm)
p.close()

Eth = dict(zip(Lm, res))

In [ ]:
table

In [ ]:
table = [[r'$L_\text{m}$', r'$E_\text{Th}$']]
for _Lm in Eth:
    table.append([_Lm, f'{Eth[_Lm][1]:.3}'])
    
print_table(table)

In [ ]:
learners = []

syst_pars = syst_pars_standard.copy()
params_raw = params_raw_standard.copy()
params = params_standard.copy()

hash_syst = hash(tuple(syst_pars))
hash_pars = hash(tuple(params_raw))
folder_path = hex(abs(hash_syst + hash_pars))
def loss(ip):
    from adaptive.learner.learner2D import default_loss, areas
    adaptive.learner.learner2D.np
    loss = default_loss(ip)
    dim = areas(ip).shape[0]
    return 1e8 * loss / dim if dim < 500 else loss    

for _Lm in Lm:
    syst_pars['Lm'] = _Lm
    
    f = partial(distributed_sns.f_adaptive, keys=('B', 'phase'), params=copy(params), 
            syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, bounds=[(0, 1.2), (-1.0*np.pi, 1.0*np.pi)], loss_per_triangle=loss)
    
    learner.pars = tuple([_Lm])
    learners.append(learner)


bl = adaptive_tools.BalancingLearner(learners)
try:
#     bl.load(folder_path)
    bl.load('0x33062dfc10a36a67')
except:
    pass

try:
    runner.cancel()
except:
    pass


# runner = adaptive.Runner(bl, executor=client)
# bl.start_periodic_saver(runner, folder_path, interval=60)
# runner.live_info()

In [ ]:
plot_dict = {Lm[i] : bl.learners[i].plot(n=500) for i in range(len(Lm))}


def B_to_Ez(B):
    return 2*B*params['g_factor']*constants['mu_B']

for _Lm in plot_dict:
    _Eth = Eth[_Lm][1]
    _bnds = plot_dict[_Lm].data[('Image', 'I')].bounds.lbrt()
    bnds = list(_bnds)
    bnds[0] = B_to_Ez(_bnds[0])
    bnds[2] = B_to_Ez(_bnds[2])
    bnds = ((bnds[0], bnds[1]), (bnds[2], bnds[3]))
    
    plot_dict[_Lm].data[('Image', 'I')].bounds.set(bnds)
    
    N_ticks = int(bnds[1][0]//_Eth)
    
    for n in range(1, N_ticks+1):
        plot_dict[_Lm] *= hv.Curve(([n*_Eth, n*_Eth], [_bnds[1], _bnds[3]]))

hv.HoloMap(plot_dict, kdims='Lm').redim(x='Ez', y='phase')


# Super current

### Define current operator

In [ ]:
params = params_standard.copy()
params['B'] = 0.45

syst_pars = syst_pars_standard.copy()
syst_pars['a'] = 12.5
syst_pars['Lm'] = 500
syst_pars['Ly'] = 1000

B_list = np.linspace(0, 2*Eth[syst_pars['Lm']][1], 21)
phase_list = np.linspace(-np.pi,np.pi, 11)
syst = sns_system.make_sns_system(**syst_pars)

In [ ]:
B_to_Ez(.45)

In [ ]:
supercurrent.fermi_dirac(0.15, params)

In [ ]:
I=dict()
pf=[]
gap=[]
for (idy, B) in enumerate(B_list):
    I[B] = list()
    for (idx, phase) in enumerate(phase_list):
        _params = params.copy()
        _params['phase'] = phase
        _params['B'] = B
        I[B].append(supercurrent.current_kpm_non_projected(syst_pars,
                                                        _params,
                                                        80,
                                                        _params['Delta']/2,
                                                        lview,
                                                        2))
    
#     _pf = topology.get_pfaffian(syst, _params)
#     pf.append(_pf)
    
    print(f'%{idx} ',end='')

In [ ]:
plot_dict = {B/Eth[250][0] : 
             hv.Curve((phase_list, np.sum(_I, axis=1)))*
             hv.Curve((phase_list, f[i](phase_list)))
             for i, (B, _I) in enumerate(I.items())}
hv.HoloMap(plot_dict, kdims='B')

In [ ]:
f = []
amplitude = []
g = lambda x, a, f, phi, o: a*np.sin(f*(x - phi)) + o
for b in B_list:
    amplitude.append(par[0])
    par, _ = scipy.optimize.curve_fit(g, phase_list, np.sum(I[b], axis=1))
    f.append(partial(g, a=par[0], f=par[1], phi=par[2], o=par[3]))

In [ ]:
Ic = []
for b,i in I.items():
    Ic.append(max(np.sum(i, axis=1)))

In [ ]:
amplitude[0]

In [ ]:
(
hv.Curve((B_list, Ic))*
hv.Curve((B_list[:-1], np.abs(amplitude[1:])))
)

In [ ]:
hv.Curve((phase_list/np.pi, np.sum(I, axis=1)))

In [ ]:
hv.Curve((phase_list/np.pi, np.sum(I, axis=1)))

In [ ]:
%lprun -f supercurrent.current_kpm_non_projected supercurrent.current_kpm_non_projected(syst_pars, _params, 60, _params['Delta']/2, lview, 2)

In [ ]:
%%output size=200
(
hv.Curve((phase_list/np.pi, np.sum(I_threight_eth, axis=1)),
         label='I_threight_eth') *    
hv.Curve((phase_list/np.pi, np.sum(I_qrt_eth, axis=1)),
         label='I_qrt_eth') *
hv.Curve((phase_list/np.pi, np.sum(I_half_eth, axis=1)),
         label='I_half_eth') *
hv.Curve((phase_list/np.pi, np.sum(I_half_eth_more_e, axis=1)),
         label='I_half_eth_more_e') *
hv.Curve((phase_list/np.pi, np.sum(I_eth, axis=1)),
         label='I_eth')
)